In [1]:
import pandas as pd


def maximimsation_marge(montant,status):
    taux_marge = 0.05
    if status == "TP":
        res = 0 # le fraudeur est détecté
    elif status == "TN":
        res = taux_marge * montant # un client honnête est accepté
    elif status == "FP":
        res = 0.7*taux_marge * montant # un client honnête est bloqué
    elif status == "FN": # un fraudeur est accepté
        if montant <= 20:
            res = 0
        elif montant <= 50:
            res = -0.2 * montant
        elif montant <= 100:
            res = -0.3 * montant
        elif montant <= 200:
            res = -0.5 * montant
        else:
            res = -0.8 * montant
    return res

In [2]:
def application_regles(y_test,y_pred):
    df = pd.DataFrame({'y_test': y_test, 'y_pred': y_pred, 'status': "null"})
    for index, row in df.iterrows():
        if row['y_test'] == 1 and row['y_pred'] == 1:
            df.at[index, 'status'] = "TP"
        elif row['y_test'] == 0 and row['y_pred'] == 0:
            df.at[index, 'status'] = "TN"
        elif row['y_test'] == 0 and row['y_pred'] == 1:
            df.at[index, 'status'] = "FP"
        elif row['y_test'] == 1 and row['y_pred'] == 0:
            df.at[index, 'status'] = "FN"
    return df

In [3]:
df_train = pd.read_csv("data//classif/dataframe_train.csv")
df_test = pd.read_csv("data//classif/dataframe_test.csv")

In [4]:
df_train.head()

,Montant,CodeDecision,VerifianceCPT1,VerifianceCPT2,VerifianceCPT3,D2CB,ScoringFP1,TauxImpNb_RB,EcartNumCheq,NbrMagasin3J,...,CA3TR,Heure,Jour,Mois,Année,Heure2,Minute,Seconde,FlagImpaye,Cluster_Code_Decision
0,20.00,1,0.0,0.0,0.0,551.0,0.0,0.0,0.0,1.0,...,0.0,27134,1,2,2017,7,32,14,0,0
1,20.00,1,0.0,0.0,0.0,551.0,0.0,0.0,1.0,2.0,...,0.0,27817,1,2,2017,7,43,37,0,0
2,57.64,1,0.0,0.0,0.0,549.0,0.0,0.0,0.0,1.0,...,0.0,28058,1,2,2017,7,47,38,0,0
3,54.29,0,1.0,1.0,1.0,267.0,0.0,0.0,0.0,1.0,...,0.0,28128,1,2,2017,7,48,48,0,0
4,26.90,1,0.0,0.0,0.0,549.0,0.0,0.0,1.0,1.0,...,0.0,29607,1,2,2017,8,13,27,0,0


In [5]:
df_test.head()

,FlagImpaye,Montant,CodeDecision,VerifianceCPT1,VerifianceCPT2,VerifianceCPT3,D2CB,ScoringFP1,TauxImpNb_RB,EcartNumCheq,...,CA3TRetMtt,CA3TR,Heure,Jour,Mois,Année,Heure2,Minute,Seconde,Cluster_Code_Decision
0,0,60.00,0,0.0,0.0,0.0,302.0,0.0,0.0,0.0,...,60.0,0.0,27257,1,9,2017,7,34,17,0
1,0,25.00,0,0.0,0.0,0.0,547.0,0.0,0.0,0.0,...,25.0,0.0,29177,1,9,2017,8,6,17,0
2,0,32.00,0,0.0,0.0,0.0,152.0,0.0,0.0,0.0,...,32.0,0.0,29283,1,9,2017,8,8,3,0
3,0,20.88,0,0.0,0.0,0.0,393.0,0.0,0.0,4.0,...,40.0,0.0,29343,1,9,2017,8,9,3,0
4,0,20.21,0,0.0,0.0,0.0,152.0,0.0,0.0,0.0,...,40.0,0.0,29497,1,9,2017,8,11,37,0


# CLASSIFICATION CODE_DECISION

In [6]:
#import pickle
# loaded_models = {}
# filename = 'models/classif_code/kmeans.pkl'
# with open(filename, 'rb') as file:
#     loaded_models["kmeans"] = pickle.load(file)



# INITIALISATION

In [14]:
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import f1_score 
import pickle

# NE pas utiliser 'CodeDecision'
X_train = df_train.drop(['FlagImpaye','CodeDecision'], axis=1)
#X_train = df_train.drop(['FlagImpaye'], axis=1)
y_train = df_train['FlagImpaye']

X_test = df_test.drop(['FlagImpaye','CodeDecision'], axis=1)
#X_test = df_test.drop(['FlagImpaye'], axis=1)
y_test = df_test['FlagImpaye']

print(X_test.columns)
print(X_train.columns)
# Modèles
models = {
    # SGD Classifier : SVM, logistic regression 
    # or
    # kernel approximation : RBFSampler
        
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Support Vector Machine': SVC(),
    'Neural Network': MLPClassifier(random_state=42),
    'Linear Discriminant Analysis': LinearDiscriminantAnalysis()
}

# Initialiser un DataFrame pour stocker les résultats
f1_df = pd.DataFrame(index=['SMOTE'], columns=models.keys())
marge_df = pd.DataFrame(index=['SMOTE'], columns=models.keys())

Index(['Montant', 'VerifianceCPT1', 'VerifianceCPT2', 'VerifianceCPT3', 'D2CB',
       'ScoringFP1', 'TauxImpNb_RB', 'EcartNumCheq', 'NbrMagasin3J',
       'DiffDateTr1', 'DiffDateTr2', 'DiffDateTr3', 'CA3TRetMtt', 'CA3TR',
       'Heure', 'Jour', 'Mois', 'Année', 'Heure2', 'Minute', 'Seconde',
       'Cluster_Code_Decision'],
      dtype='object')
Index(['Montant', 'VerifianceCPT1', 'VerifianceCPT2', 'VerifianceCPT3', 'D2CB',
       'ScoringFP1', 'TauxImpNb_RB', 'EcartNumCheq', 'NbrMagasin3J',
       'DiffDateTr1', 'DiffDateTr2', 'DiffDateTr3', 'CA3TRetMtt', 'CA3TR',
       'Heure', 'Jour', 'Mois', 'Année', 'Heure2', 'Minute', 'Seconde',
       'Cluster_Code_Decision'],
      dtype='object')


# TRAINING

In [8]:
# Train chaque modèle
for model_name, model in models.items():
    
    # poids de classes ? normalement pas besoin car on fait f1(weighted) ???
    #class_weights = {0: 1, 1: 5} 
    # Entraîner le modèle en spécifiant les poids des classes

    model.fit(X_train, y_train)
    print(model_name, " fitted")

    # save models
    filename = 'models/' + model_name + '.pkl'
    with open(filename, 'wb') as file:
        pickle.dump(model, file)

Decision Tree  fitted


# LOAD MODELS & PREDICT

In [15]:
loaded_models = {}
for model_name in models.keys():
    filename = 'models/' + model_name + '.pkl'
    with open(filename, 'rb') as file:
        loaded_models[model_name] = pickle.load(file)

# Test chaque modèle
for model_name, model in loaded_models.items():
    y_pred = model.predict(X_test)
    #print(y_pred.shape)
    #print(y_test.shape)
    f1 = f1_score(y_test, y_pred, average='weighted')
    f1_df.at['SMOTE', model_name] = f1

        
    df_res = application_regles(y_test,y_pred)
    for row in df_res.itertuples():
        df_res.at[row.Index, 'marge'] = maximimsation_marge(X_test.at[row.Index, 'Montant'],row.status)

    total_marge = df_res['marge'].sum()
    marge_df.at['SMOTE', model_name] = total_marge

    print("\ F1 score du modèle sur l'ensemble de test :", f1)
    print(confusion_matrix(y_test, y_pred))

\ F1 score du modèle sur l'ensemble de test : 0.9529313992460843
[[690739  50099]
 [  5762    811]]


In [16]:
marge_df.to_csv('data/smote/marge.csv')
marge_df.head()

,Decision Tree
SMOTE,1927906.39825


In [17]:
f1_df.to_csv('data/smote/f1_df.csv')
f1_df.head()

,Decision Tree
SMOTE,0.952931


In [18]:
df_train["FlagImpaye"].value_counts()

FlagImpaye
0    3875940
1    3875940
Name: count, dtype: int64

In [19]:
df_test["FlagImpaye"].value_counts()

FlagImpaye
0    740838
1      6573
Name: count, dtype: int64

In [ ]:
# changer TP en => TP = FRAUDE = prédit fraude, car sinon c'est le mauvais f1 score


#optimisation de la marge
#  donner la marge maximale (si modèle est parfait) et comparer avec la meilleure marge obtenue, du genre 'on a perdu 10% de marge par rapport au modèle parfait'